In [1]:
import sys
sys.path.append('..')

import os
import random
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
from model.model_lora import *
from trainer.trainer_utils import setup_seed
import pandas as pd
import json
from types import SimpleNamespace

In [2]:
DATA_LOC = '/root/xlcoder/MiniMind2-Small/dataset'
INSTR_LOC = '/root/xlcoder/MiniMind2-Small/hands-on'

In [3]:
def init_model(args):
    tokenizer = AutoTokenizer.from_pretrained(args.load_from)
    if 'model' in args.load_from:
        model = MiniMindForCausalLM(MiniMindConfig(
            hidden_size=args.hidden_size,
            num_hidden_layers=args.num_hidden_layers,
            use_moe=bool(args.use_moe),
            inference_rope_scaling=args.inference_rope_scaling
        ))
        moe_suffix = '_moe' if args.use_moe else ''
        ckp = f'./{args.save_dir}/{args.weight}_{args.hidden_size}{moe_suffix}.pth'
        model.load_state_dict(torch.load(ckp, map_location=args.device), strict=True)
        if args.lora_weight != 'None':
            apply_lora(model)
            load_lora(model, f'./{args.save_dir}/lora/{args.lora_weight}_{args.hidden_size}.pth')
    else:
        model = AutoModelForCausalLM.from_pretrained(args.load_from, trust_remote_code=True)
    print(f'MiniMind模型参数: {sum(p.numel() for p in model.parameters()) / 1e6:.2f} M(illion)')
    return model.eval().to(args.device), tokenizer

In [8]:
args={
    # 'load_from':'/root/xlcoder/MiniMind2-Small/MiniMind2',
    'load_from': '../model',
    'save_dir': '../out',
    # 'weight': 'full_sft',
    'weight': 'pretrain',
    'lora_weight': 'None',
    # 'hidden_size': 768,
    'hidden_size': 512,
    # 'num_hidden_layers': 16,
    'num_hidden_layers': 8,
    'use_moe': 0,
    'inference_rope_scaling': False,
    'max_new_tokens':8192,
    'temperature':0.85,
    'top_p':0.85,
    'historys':0,
    'device':'cuda' if torch.cuda.is_available() else 'cpu',
    'times': 1,
    'count_only': False
}
args = json.loads(json.dumps(args), object_hook=lambda d: SimpleNamespace(**d))

In [5]:
raw_data = pd.read_csv(os.path.join(DATA_LOC, 'bbc-news-data.csv'), sep='\t')
raw_data

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [6]:
to_process = raw_data.iloc[:10]
to_process;

In [11]:
model, tokenizer = init_model(args)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
with open(os.path.join(INSTR_LOC, 'few_shot1_pre.txt'), 'r') as pre_file:
    pre_instruct = pre_file.read()

with open(os.path.join(INSTR_LOC, 'few_shot1_post.txt'), 'r') as post_file:
    post_instruct = post_file.read()
    
for _ in range(int(args.times)):
    prompt_iter = raw_data.sample(n=10)
    # pre_instruct = 
    # post_instruct = "Your answer (choose ONLY one category in business, entertainment, politics, sport, tech):\n"
    for i, prompt in prompt_iter.iterrows():
        # setup_seed(2026) # or setup_seed(random.randint(0, 2048))
        prompt = pre_instruct+'title:\n'+prompt['title']+'\ncontent:\n'+prompt['content']+post_instruct
        # prompt = '您好，请问有什'
        setup_seed(random.randint(0, 2048))
        conversation = conversation[-args.historys:] if args.historys else []
        # conversation.append({"role": "user", "content": prompt})
        # conversation.append({"role":"user", "content": "今天天气怎么样？"})
        # print(conversation)
        templates = {"conversation": conversation, "tokenize": False, "add_generation_prompt": True}
        # if args.weight == 'reason': templates["enable_thinking"] = True # 仅Reason模型使用
        inputs = tokenizer.apply_chat_template(**templates) if args.weight != 'en_pretrain' else (tokenizer.bos_token + prompt)
        inputs = tokenizer(inputs, return_tensors="pt", truncation=True).to(args.device)

        print('🤖️: ', end='')
        generated_ids = model.generate(
            inputs=inputs["input_ids"], attention_mask=inputs["attention_mask"],
            max_new_tokens=args.max_new_tokens, do_sample=True, streamer=streamer,
            pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id,
            top_p=args.top_p, temperature=args.temperature, repetition_penalty=1.0
        )
        response = tokenizer.decode(generated_ids[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)
        if args.count_only:
            res_len=len(response)
            # ngrm_rep = ngram_repetition(response, 4)
            ngrm_rep = weighted_ngram_local_activation(response, 4, normalize='empirical')
            print("token_count:", res_len, "weighted_4-gram_repitition:", ngrm_rep)
            df.append({'prompt_id':i,'token_count':len(response), 'weighted_4-gram_repitition': ngrm_rep})
            continue
        conversation.append({"role": "assistant", "content": response})
        print('\n\n')

MiniMind模型参数: 25.83 M(illion)
🤖️: 麼方法可以提高我們的網遊遊戲？網遊遊戲有很多方法，可以提高我們的網遊遊戲，從日常的資源出發，並且進行有效的參與和交流，例如聯繫網站，銷售服務，遊戲詳情推薦遊戲，參賽網絡等等。此外，還可以多元化的遊戲設施，以及多媒體活動來減少問題，並在家裡從遊戲中遠入。最後，網遊遊帶有著不同的技術和實力，你可以考慮到不同地域的遊戲策略，以提高對不同環境的遊戲過程。



🤖️: 麼可以帮助我克服拖延症？拖延症是一種非常有效的方法，因為它可以幫助我克服拖延症。這種方法將我的組成一致，將它來自於您的技能和行動，以及您對這些事情的責任。如果您想讓自己進一步改進，我很容易被擔心，而且對我們的成長有很大的影響。



🤖️: 麼可以進一步了解車輛的安全問題？車輛安全問題是一個關鍵的問題，它們需要考慮安全，例如如何保障駕駛安全，如何防止車輛運行及車輛的安全。另外，車輛的安全問題也非常重要，需要將車輛安全地處理到車輛，避免車輛過度發生危險。導致車輛運行時，車輛的安全問題也很重要，因此需要在車輛安全地處理車輛，避免車輛運行。此外，導致車輛的安全問題也非常重要，需要車輛的安全計劃和安全措施。



🤖️: 麼方式可以降低碳排放量？是的，可以降低碳排放量，是降低碳排放量的方法之一。例如，可以使用太阳能和风能等可再生能源，减少对化石燃料的依赖；可以采用清洁能源，如风能、太阳能等，来降低碳排放。此外，也可以使用可再生能源，如太阳能和风能，来减少对化石燃料的依赖。



🤖️: 麼方法可以提高我的決策能力？我們可以通過統計和計算，讓你對現有數據的理解能力進行分析和應用。此外，我們也可以提供有關鍵監督和數據分析等技術，以改善模型的性能和性能。最重要的是，我們需要更深入的研究和理解，以確保模型的可信度。



🤖️: 麼方法可以提高您的禮物質量？給你一些方法，可以提高組織組織的生產力，包括提高組織品位、減少資源浪費、提高組織品位、提升品質等等。此外，維持組織結構和市場合紀的模型，也是組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織組織